# Disease Tagging Tutorial

In this example, we'll be writing an application to extract *mentions of* diseases from Pubmed abstracts, using annotations from the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial, which has 5 parts, walks through the process of constructing a model to classify _candidate_ disease mentions as either true (i.e., that it is truly a mention of a disease) or false.

## Part V: Evaluating the Model on the Test Set

In this final part of the tutorial, we will reload the model developed in Part IV and test it on the test `CandidateSet`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# Note: We run automated tests on this tutorial to make sure that it is always up to date! 
# However, certain interactive components cannot currently be tested automatically, and will 
# be skipped with if-then statements using the variable below
AUTOMATED_TESTING = os.environ.get('TESTING') is not None

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Disease` `Candidate` subclass from Parts II, III, and IV.

In [ ]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('Disease', ['disease'])

## Loading Test `CandidateSet`

We reload the test `CandidateSet` object from the previous parts of the tutorial.

In [ ]:
from snorkel.models import CandidateSet

test = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates').one()

## Automatically Creating Features
We also generate features for the test set.

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

**_NOTE: This may take ~1 hour currently_**

In [ ]:
%time F_test = feature_manager.update(session, test, 'Train Features', False)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_test = feature_manager.load(session, test, 'Train Features')

## Reloading the Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
%time disc_model.load(session, "Discriminative Params")

## Evaluating on the Development `CandidateSet`

First, we load the development set labels and gold candidates we made in Part III.

In [ ]:
L_test = label_manager.load(session, dev, "CDR Test Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates -- Gold').one()

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

## Viewing Examples
After evaluating on the test `CandidateSet`, we can again view examples in the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
if not AUTOMATED_TESTING:
    sv = SentenceNgramViewer(tp[:300], session, annotator_name="Tutorial Part V User")
else:
    sv = None

In [ ]:
sv

You've completed the tutorial!